# <center> Introduction to Spark In-memmory Computing via Spark Scala </center>

- Spark is an implementation of the MapReduce programming paradigm that operates on in-memory data and allows data reuses across multiple computations.
- Performance of Spark is significantly better than its predecessor, Hadoop MapReduce. 
- Spark's primary data abstraction is Resilient Distributed Dataset (RDD):
    - Read-only, partitioned collection of records
    - Created (aka written) through deterministic operations on data:
        - Loading from stable storage
        - Transforming from other RDDs
        - Generating through coarse-grained operations such as map, join, filter ...
    - Do not need to be materialized at all time and are recoverable via **data lineage**

<img src="pictures/18/spark2_arch.png" width="600"/>

SSH into CloudLab. 

```
$ ssh node218.clemson.cloudlab.us
```

From inside the terminal, open Spark's interactive shell

```
$ spark-shell --master yarn --driver-memory 1G --executor-memory 10G --num-executors 10 --verbose
``` 

## 1. Getting Started

Spark stores data in memory. This memory space is represented by variable **sc** (SparkContext). 

The spark shell's prompt is **scala>**

```
scala> sc
```

```
val textFile = sc.textFile("hdfs:///repository/gutenberg-shakespeare.txt")
```

What type is `textFile`?

```
scala> print (textFile)
scala> printf (textFile)
```

## 2. What does Spark do with my data?

**Storage Level:**
- Does RDD use disk?
- Does RDD use memory?
- Does RDD use off-heap memory?
- Should an RDD be serialized (while persisting)?
- How many replicas (default: 1) to use?

```
scala> textFile.getStorageLevel.useDisk
scala> textFile.getStorageLevel.useMemory
scala> textFile.getStorageLevel.useOffHeap
scala> textFile.getStorageLevel.deserialized
scala> textFile.getStorageLevel.replication
```

Set cache to Spark:

```
scala> textFile.cache
scala> textFile.getStorageLevel
```

- By default, each transformed RDD may be recomputed each time you run an action on it.
- It is also possible to *persist* RDD in memory using *persist()* or *cache()*
    - *persist()* allows you to specify level of storage for RDD
    - *cache()* only persists RDD in memory
    - To retire RDD from memory, *unpersist()* is called

## 3. WordCount

Data operations in Spark are categorized into two groups, *transformation* and *action*. 
- A *transformation* creates new dataset from existing data. Examples of *transformation* include map, filter, reduceByKey, and sort. 
- An *action* returns a value to the driver program (aka memory space of this notebook) after running a computation on the data set. Examples of *action* include count, collect, reduce, and save. 

"All transformations in Spark are lazy, in that they do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result to be returned to the driver program." -- Spark Documentation

#### RDD Operations in Spark

**Transformations: **

- *map*(f: T -> U) : RDD[T] -> RDD[U]
- *filter*(f: T -> Bool) : RDD[T] -> RDD[T]
- *flatMap*(f: T -> Seq[U]) : RDD[T] -> RDD[U]
- *sample*(*fraction*: Float) : RDD[T] -> RDD[T] (deterministic sampling)
- *groupByKey*() : RDD[(K,V)] -> RDD[(K, Seq[V])]
- *reduceByKey*(f: (V,V) -> V) : RDD[(K,V)] -> RDD[(K,V)]
- *union*() : (RDD[T], RDD[T]) -> RDD[T]
- *join*() : (RDD[(K,V)], RDD[(K,W)]) -> RDD[(K,(V,W))]
- *cogroup*() : (RDD[(K,V)], RDD[(K,W)] -> RDD[(K, (Seq[V],Seq[W]))]
- *crossProduct*() : (RDD[T], RDD[U]) -> RDD[(T,U)]
- *mapValues*(f: V -> W) : RDD[(K,V)] -> RDD[(K,W)] (preserves partitioning)
- *sort*(c: Comparator[K]) :  RDD[(K,V)] -> RDD[(K,V)]
- *partitionBy*(p: Partitioner[K]) : RDD[(K,V)] -> RDD[(K,V)]

**Actions:**

- *count*() : RDD[T] -> Long
- *collect*() : RDD[T] -> Seq[T]
- *reduce*(f: (T,T) -> T) : RDD[T] -> T
- *lookup*(k : K) : RDD[(K,V)] -> Seq[V] (on hash/range partitionied RDDs)
- *save*(path: String) : Outputs RDD to a storage system 

#### Cache versus Cached

Count how many lines there are in the file

```
scala> spark.time(textFile.count)
```

Recount

```
scala> spark.time(textFile.count)
```

How to run wordcount

```
scala> val wordcount = textFile.flatMap(line => line.split(" ")).map(word => (word, 1)).reduceByKey{case (x, y) => x + y}
```

In [15]:
wordcount

PythonRDD[9] at RDD at PythonRDD.scala:48

In [16]:
!hdfs dfs -rm -r intro-to-spark
!hdfs dfs -mkdir intro-to-spark

17/11/01 12:44:02 INFO fs.TrashPolicyDefault: Moved: 'hdfs://dsci/user/lngo/intro-to-spark' to trash at: hdfs://dsci/user/lngo/.Trash/Current/user/lngo/intro-to-spark


In [17]:
wordcount.saveAsTextFile("intro-to-spark/output-wordcount-01")

In [18]:
!hdfs dfs -cat intro-to-spark/output-wordcount-01/part-00000 \
    2>/dev/null | head -n 20

('', 516839)
('Quince', 1)
('LIBRARY,', 218)
('Just', 10)
('enrooted', 1)
('divers', 20)
('Doubtless', 2)
('undistinguishable,', 1)
('Rheims,', 1)
('Freedom!', 1)
('incorporate.', 1)
('bawd!', 3)
('Sir-I', 1)
('withering', 2)
('Mopsa,', 1)
('[BEROWNE', 3)
('forgetfulness?', 1)
('Tranio?', 1)
('Wound', 3)
('twice,', 2)


**Step-by-step actions:**

In [19]:
!hdfs dfs -cat /repository/gutenberg-shakespeare.txt \
    2>/dev/null | head -n 100

1609

THE SONNETS

by William Shakespeare



                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy l

In [20]:
wordcount_step_01 = textFile.flatMap(lambda line: line.split(" "))

In [21]:
wordcount_step_01

PythonRDD[13] at RDD at PythonRDD.scala:48

In [22]:
wordcount_step_01.take(20)

['1609',
 '',
 'THE',
 'SONNETS',
 '',
 'by',
 'William',
 'Shakespeare',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [23]:
wordcount_step_02 = wordcount_step_01.map(lambda word: (word, 1))

In [24]:
wordcount_step_02.take(20)

[('1609', 1),
 ('', 1),
 ('THE', 1),
 ('SONNETS', 1),
 ('', 1),
 ('by', 1),
 ('William', 1),
 ('Shakespeare', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1),
 ('', 1)]

In [25]:
wordcount_step_03 = wordcount_step_02.reduceByKey(lambda a, b: a + b)

In [26]:
wordcount_step_03.take(20)

[('', 516839),
 ('Quince', 1),
 ('LIBRARY,', 218),
 ('Just', 10),
 ('enrooted', 1),
 ('divers', 20),
 ('Doubtless', 2),
 ('undistinguishable,', 1),
 ('Rheims,', 1),
 ('Freedom!', 1),
 ('incorporate.', 1),
 ('bawd!', 3),
 ('Sir-I', 1),
 ('withering', 2),
 ('Mopsa,', 1),
 ('[BEROWNE', 3),
 ('forgetfulness?', 1),
 ('Tranio?', 1),
 ('Wound', 3),
 ('twice,', 2)]

### Challenge

- Augment the mapping process of WordCount with a function to filter out punctuations and capitalization from the unique words

To stop the Spark job, call `sc.stop()`

In [27]:
sc.stop()